In [1]:
import h2o
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 hour 6 mins
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.3
H2O_cluster_version_age:,6 days
H2O_cluster_name:,H2O_from_python_avinash_xiu7lj
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.033 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [2]:
games = h2o.import_file('train.csv')
test = h2o.import_file('test.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [3]:
c_score_col = ['ScoreDiffNorm', 'Pred', 'SeedDiff', 'NumOT',
 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF',
 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF' ,
 'WEFFG', 'WEFFG3', 'WDARE', 'WTOQUETOQUE', 'LEFFG', 'LEFFG3', 'LDARE', 'LTOQUETOQUE',
 'DFGM', 'DFGA', 'DFGM3', 'DFGA3', 'DFTM', 'DFTA', 'DOR', 'DDR', 'DAst', 'DTO', 'DStl', 'DBlk', 'DPF']
col = [c for c in games.columns if c not in ['ID', 'DayNum', 'ST', 'Team1', 'Team2', 'IDTeams', 'IDTeam1', 'IDTeam2', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'NumOT', 'Pred', 'ScoreDiff', 'ScoreDiffNorm', 'WLoc', 'IDTeams_c_score']+c_score_col]
len(col)

360

In [4]:
x = col
y = "Pred"

# For binary classification, response should be a factor
games[y] = games[y].asfactor()

In [6]:
train, valid = games.split_frame(ratios=[.7])
train.shape, valid.shape

((46071, 426), (19801, 426))

In [7]:
from h2o.automl import H2OAutoML
aml = H2OAutoML(sort_metric='logloss', max_models=2, include_algos=["GLM", "DRF", "XGBoost"], seed=1234)
aml.train(x=x, y=y, training_frame=train, validation_frame=valid)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)

AutoML progress: |
17:45:59.595: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

███████████████████████████████████████████████████████████████| (done) 100%


model_id,logloss,auc,aucpr,mean_per_class_error,rmse,mse
GLM_1_AutoML_2_20220223_174559,0.466908,0.839341,0.844161,0.27172,0.401926,0.161545
XGBoost_1_AutoML_2_20220223_174559,0.564447,0.78862,0.798873,0.338289,0.442732,0.196011


In [9]:
from sklearn.metrics import log_loss

In [12]:
pred = aml.predict(train[x]).as_data_frame()['p1'].to_list()
print('Log Loss:', log_loss(train.as_data_frame()[y].astype(float), pred))

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Log Loss: 0.45909406759157917


In [11]:
pred = aml.predict(valid[x]).as_data_frame()['p1'].to_list()
print('Log Loss:', log_loss(valid.as_data_frame()[y].astype(float), pred))

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Log Loss: 0.45816793239995074


In [13]:
import pandas as pd
correct_file= pd.read_csv('perfect_submission.csv')
idx = (correct_file.Pred!=0.5) #& (correct_file['ID'].str.startswith('2016'))
true_pred = correct_file[idx]

In [14]:
test_df= test.as_data_frame()

In [58]:
test_pred = aml.predict(test[x]).as_data_frame()['p1']
test_df['Pred'] = test_pred
test_df['Pred'] = test_df.Pred.clip(0.00001, 0.99999)
print('Log Loss:', log_loss(true_pred['Pred'], test_df.loc[idx, 'Pred']))

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Log Loss: 1.9099187856905278


In [33]:
test_df[['ID','Pred']].to_csv('h2o_submission.csv', index=False)